In [9]:
import os
os.getcwd() 
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import warnings
warnings.filterwarnings("ignore")
from T5 import *
import torch
from datasets import load_dataset,load_metric
from transformers import T5Tokenizer,AutoModelForSeq2SeqLM, AutoTokenizer
import torch_optimizer as optim
from transformers.optimization import Adafactor, AdafactorSchedule
from MT_hyperparams import seed_,max_length,target_language
import torch.backends.cudnn as cudnn
from utils import *
from attention_params import *
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler
from torch.autograd import Variable
from losses import *
from architect import *
import logging
import sys
import transformers
import time
import argparse
from tqdm import tqdm
import string
from torch.optim.lr_scheduler import LambdaLR

from torch.optim.lr_scheduler import StepLR

In [2]:
parser = argparse.ArgumentParser("main")


parser.add_argument('--valid_num_points', type=int,             default = 100, help='validation data number')
parser.add_argument('--train_num_points', type=int,             default = 200, help='train data number')

parser.add_argument('--batch_size', type=int,                   default=16,     help='Batch size')
parser.add_argument('--train_w_num_points', type=int,           default=16,      help='train_w_num_points for each batch')
parser.add_argument('--train_v_synthetic_num_points', type=int, default=0,      help='train_v_synthetic_num_points for each batch')
parser.add_argument('--train_v_num_points', type=int,           default=0,      help='train_v_num_points for each batch')
parser.add_argument('--train_A_num_points', type=int,           default=0,      help='train_A_num_points decay for each batch')


parser.add_argument('--gpu', type=int,                          default=0,      help='gpu device id')
parser.add_argument('--model_name_teacher', type=str,           default='google/t5-small-lm-adapt',      help='model_name')
parser.add_argument('--model_name_student', type=str,           default='t5-small',      help='model_name')
parser.add_argument('--exp_name', type=str,                     default='T5spec',      help='experiment name')
parser.add_argument('--rep_num', type=int,                      default=50,      help='report times for 1 epoch')
parser.add_argument('--test_num', type=int,                     default=200,      help='test times for 1 epoch')

parser.add_argument('--epochs', type=int,                       default=500,     help='num of training epochs')
parser.add_argument('--pre_epochs', type=int,                   default=0,      help='train model W for x epoch first')
parser.add_argument('--grad_clip', type=float,                  default=1,      help='gradient clipping')
parser.add_argument('--grad_acc_count', type=float,             default=-1,      help='gradient accumulate steps')

parser.add_argument('--w_lr', type=float,                       default=1e-3,   help='learning rate for w')
parser.add_argument('--unrolled_w_lr', type=float,              default=0,   help='learning rate for w')
parser.add_argument('--v_lr', type=float,                       default=0,   help='learning rate for v')
parser.add_argument('--unrolled_v_lr', type=float,              default=0,   help='learning rate for v')
parser.add_argument('--A_lr', type=float,                       default=0,   help='learning rate for A')
parser.add_argument('--learning_rate_min', type=float,          default=1e-8,   help='learning_rate_min')
parser.add_argument('--decay', type=float,                      default=1e-3,   help='weight decay')
parser.add_argument('--beta1', type=float,                      default=0.9,    help='momentum')
parser.add_argument('--beta2', type=float,                      default=0.98,    help='momentum')
parser.add_argument('--warm', type=float,                       default=10,    help='warmup step')
# parser.add_argument('--smoothing', type=float,                  default=0.1,    help='labelsmoothing')


parser.add_argument('--traindata_loss_ratio', type=float,       default=0,    help='human translated data ratio')
parser.add_argument('--syndata_loss_ratio', type=float,         default=1,    help='augmented dataset ratio')

parser.add_argument('--valid_begin', type=int,                  default=1,      help='whether valid before train')
parser.add_argument('--train_A', type=int,                      default=1 ,     help='whether train A')



args = parser.parse_args(args=[])#(args=['--batch_size', '8',  '--no_cuda'])#used in ipynb
args.test_num = args.test_num//args.batch_size * args.batch_size
args.rep_num = args.rep_num//args.batch_size * args.batch_size

In [3]:
#https://wandb.ai/ check the running status online
import wandb
os.environ['WANDB_API_KEY']='a166474b1b7ad33a0549adaaec19a2f6d3f91d87'
os.environ['WANDB_NAME']=args.exp_name
os.environ["TOKENIZERS_PARALLELISM"] = "false"
wandb.init(project="Selftraining",config=args)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: onlydrinkwater. Use `wandb login --relogin` to force relogin


In [4]:
# logging file
now = time.strftime("%Y-%m-%d-%H_%M_%S", time.localtime(time.time()))

log_format = '%(asctime)s |\t  %(message)s'
logging.basicConfig(stream=sys.stdout, level=logging.INFO,
                    format=log_format, datefmt='%m/%d %I:%M:%S %p')
fh = logging.FileHandler(os.path.join(
    "./log/", now+'.txt'), 'w', encoding="UTF-8")
fh.setFormatter(logging.Formatter(log_format))
logging.getLogger().addHandler(fh)
dataset = load_dataset('wmt14', 'de-en')

logging.info(args)
logging.info(dataset)
logging.info(dataset['train'][5])


# Setting the seeds
np.random.seed(seed_)
torch.cuda.set_device(args.gpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cudnn.benchmark = True
torch.manual_seed(seed_)
cudnn.enabled = True
torch.cuda.manual_seed(seed_)


06/12 11:40:46 PM |	  Reusing dataset wmt14 (C:\Users\kevin\.cache\huggingface\datasets\wmt14\de-en\1.0.0\d239eaf0ff090d28da19b6bc9758e24634d84de0a1ef092f0b5c54e6f132d7e2)


100%|██████████| 3/3 [00:00<00:00,  5.62it/s]

06/12 11:40:47 PM |	  Namespace(A_lr=0, batch_size=16, beta1=0.9, beta2=0.98, decay=0.001, epochs=500, exp_name='T5spec', gpu=0, grad_acc_count=-1, grad_clip=1, learning_rate_min=1e-08, model_name_student='t5-small', model_name_teacher='google/t5-small-lm-adapt', pre_epochs=0, rep_num=48, syndata_loss_ratio=1, test_num=192, train_A=1, train_A_num_points=0, train_num_points=200, train_v_num_points=0, train_v_synthetic_num_points=0, train_w_num_points=16, traindata_loss_ratio=0, unrolled_v_lr=0, unrolled_w_lr=0, v_lr=0, valid_begin=1, valid_num_points=100, w_lr=0.001, warm=10)
06/12 11:40:47 PM |	  DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 4508785
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3003
    })
})
06/12 11:40:47 PM |	  {'translation': {'de': 'Ich bitte Sie, sich zu einer Schweigeminute zu erheben.', 'en': "Ple

In [5]:
# def randomize_model(model):
#     for module_ in model.named_modules(): 
#         # print('!',type(module_[1]))
#         if isinstance(module_[1],(torch.nn.Linear, torch.nn.Embedding)):
#             module_[1].weight.data.normal_(mean=0.0, std=0.001)
#         elif isinstance(module_[1], transformers.models.t5.modeling_t5.T5LayerNorm):
#             module_[1].weight.data.fill_(0.1)
#         if isinstance(module_[1], torch.nn.Linear) and module_[1].bias is not None:
#             module_[1].bias.data.zero_()
#     return model

In [6]:
# from transformers import AutoTokenizer, GPT2LMHeadModel, T5ForConditionalGeneration, T5Config
# config = T5Config.from_pretrained('t5-small')
# print(config)
# model = T5ForConditionalGeneration(config)
# model = randomize_model(model)
# model_size = sum(t.numel() for t in model.parameters())
# print(f"T5 size: {model_size/1000**2:.1f}M parameters")
# modelname = 't5-small'
# torch.save(model,modelname+'.pt')

In [5]:
modelname = args.model_name_teacher
pretrained  =  AutoModelForSeq2SeqLM.from_pretrained(modelname)
pathname = modelname.replace('/','')
logging.info(f'modelsize:{count_parameters_in_MB(pretrained)}MB')
torch.save(pretrained,pathname+'.pt')

modelname = args.model_name_student
pretrained  =  AutoModelForSeq2SeqLM.from_pretrained(modelname)
pathname = modelname.replace('/','')
logging.info(f'modelsize:{count_parameters_in_MB(pretrained)}MB')
torch.save(pretrained,pathname+'.pt')

06/12 11:40:48 PM |	  modelsize:76.961152MB
06/12 11:40:50 PM |	  modelsize:60.506624MB


In [6]:

# preprocess the data, make a dataloader
import random
modelname = args.model_name_teacher
tokenizer = AutoTokenizer.from_pretrained(modelname)
criterion = torch.nn.CrossEntropyLoss( reduction='none')#teacher shouldn't have label smoothing, especially when student got same size.
criterion_v = torch.nn.CrossEntropyLoss( reduction='none')#,label_smoothing=args.smoothing) #without LS, V may be too confident to that syn data, and LS do well for real data also.
# dataset = dataset.shuffle(seed=seed_)
train = dataset['train']['translation'][:args.train_num_points]
valid = dataset['train']['translation'][:args.valid_num_points]#TODO:change dataset['validation']['translation'][:args.valid_num_points]args.train_num_points:args.train_num_points+args.valid_num_points
test = dataset['test']['translation']#[L_t+L_v:L_t+L_v+L_test]

def preprocess(dat):
    for t in dat:
        t['en'] = "translate English to German: " + t['en']  #needed for T5
preprocess(train)
preprocess(valid)
# preprocess(test)#TODO:
#TODO: Syn_input should be monolingual data, should try en-fo's en. cuz wmt may align
num_batch = args.train_num_points//args.batch_size
train = train[:args.batch_size*num_batch]
logging.info("train len: %d",len(train))

'''
each mini batch consist of : 
1. data to train W
2. monolingual data to generate parallel data
3. data to train V
4. data to train A
'''
train_w_num_points_len = num_batch * args.train_w_num_points
train_v_synthetic_num_points_len = num_batch * args.train_v_synthetic_num_points
train_v_num_points_len = num_batch * args.train_v_num_points
train_A_num_points_len = num_batch * args.train_A_num_points
logging.info("train_w_num_points_len: %d",train_w_num_points_len)
logging.info("train_v_synthetic_num_points_len: %d",train_v_synthetic_num_points_len)
logging.info("train_v_num_points_len: %d",train_v_num_points_len)
logging.info("train_A_num_points_len: %d",train_A_num_points_len)

attn_idx_list = torch.arange(train_w_num_points_len).cuda()
logging.info("valid len: %d",len(valid))
logging.info("test len: %d" ,len(test))
logging.info(train[2])
logging.info(valid[2])
logging.info(test[2])

Downloading: 100%|██████████| 1.33M/1.33M [00:00<00:00, 3.30MB/s]


06/12 11:42:17 PM |	  train len: 192
06/12 11:42:17 PM |	  train_w_num_points_len: 192
06/12 11:42:17 PM |	  train_v_synthetic_num_points_len: 0
06/12 11:42:17 PM |	  train_v_num_points_len: 0
06/12 11:42:17 PM |	  train_A_num_points_len: 0
06/12 11:42:18 PM |	  valid len: 100
06/12 11:42:18 PM |	  test len: 3003
06/12 11:42:18 PM |	  {'de': 'Wie Sie feststellen konnten, ist der gefürchtete "Millenium-Bug " nicht eingetreten. Doch sind Bürger einiger unserer Mitgliedstaaten Opfer von schrecklichen Naturkatastrophen geworden.', 'en': "translate English to German: Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful."}
06/12 11:42:18 PM |	  {'de': 'Wie Sie feststellen konnten, ist der gefürchtete "Millenium-Bug " nicht eingetreten. Doch sind Bürger einiger unserer Mitgliedstaaten Opfer von schrecklichen Naturkatastrophen geworden.', 'en': "translate En

In [7]:
target_language  = 'de'
train_data = get_train_Dataset(train, tokenizer)# Create the DataLoader for our training set.
logging.info('train data get')
train_dataloader = DataLoader(train_data, sampler= SequentialSampler(train_data), 
                        batch_size=args.batch_size, pin_memory=False, num_workers=0)
logging.info('train data loader get')
valid_data = get_aux_dataset(valid, tokenizer)# Create the DataLoader for our training set.
valid_dataloader = DataLoader(valid_data, sampler=SequentialSampler(valid_data), 
                        batch_size=args.batch_size, pin_memory=False, num_workers=0)
logging.info('valid data loader get')
test_data = get_aux_dataset(test, tokenizer)# Create the DataLoader for our training set.
test_dataloader = DataLoader(test_data, sampler=SequentialSampler(test_data),
                        batch_size=args.batch_size, pin_memory=False, num_workers=0)#, sampler=RandomSampler(test_data)
logging.info('test data loader get')

get train data start
get train data end
06/12 11:42:18 PM |	  train data get
06/12 11:42:18 PM |	  train data loader get
06/12 11:42:18 PM |	  valid data loader get
06/12 11:42:19 PM |	  test data loader get


In [10]:

A = attention_params(train_w_num_points_len)#half of train regarded as u
A = A.cuda()



# TODO: model loaded from saved model
model_w = T5(criterion=criterion, tokenizer= tokenizer, args = args, name = 'model_w_in_main')
model_w = model_w.cuda()
w_optimizer = torch.optim.Adam(model_w.parameters(),  lr= args.w_lr ,  betas=(args.beta1, args.beta2) ,eps=1e-9 )
# w_optimizer = Adafactor(model_w.parameters(), lr = args.w_lr ,scale_parameter=False, relative_step=False , warmup_init=False,clip_threshold=1,beta1=0,eps=( 1e-30,0.001))
scheduler_w  =   StepLR(w_optimizer, step_size=1, gamma=0.5)
# scheduler_w  = Scheduler(w_optimizer,dim_embed=512, warmup_steps=args.warm, initlr = args.w_lr)



model_v = T5(criterion=criterion_v, tokenizer= tokenizer, args = args, name = 'model_v_in_main')
model_v = model_v.cuda()
v_optimizer = torch.optim.Adam(model_v.parameters(),  lr= args.v_lr ,  betas=(args.beta1,args.beta2) ,eps=1e-9  )
# v_optimizer =Adafactor(model_v.parameters(), lr = args.v_lr ,scale_parameter=False, relative_step=False , warmup_init=False,clip_threshold=1,beta1=0,eps=( 1e-30,0.001))
scheduler_v  = Scheduler(v_optimizer,dim_embed=512, warmup_steps=args.warm, initlr = args.v_lr)


architect = Architect(model_w, model_v,  A, args)

In [11]:
@torch.no_grad()
def my_test(_dataloader,model,epoch):
    # logging.info(f"GPU mem before test:{getGPUMem(device)}%")
    acc = 0
    counter = 0
    model.eval()
    metric_sacrebleu =  load_metric('sacrebleu')
    # metric_bleu =  load_metric('bleu')
    
    for step, batch in enumerate(_dataloader):
        
        test_dataloaderx = Variable(batch[0], requires_grad=False).to(device, non_blocking=False)[:args.train_w_num_points]
        test_dataloaderx_attn = Variable(batch[1], requires_grad=False).to(device, non_blocking=False)[:args.train_w_num_points]
        test_dataloadery = Variable(batch[2], requires_grad=False).to(device, non_blocking=False)[:args.train_w_num_points]
        test_dataloadery_attn = Variable(batch[3], requires_grad=False).to(device, non_blocking=False)[:args.train_w_num_points]
        ls = my_loss(test_dataloaderx,test_dataloaderx_attn,test_dataloadery,test_dataloadery_attn,model)
        acc+= ls.item()
        counter+= 1
        pre = model.generate(test_dataloaderx)
        x_decoded = tokenizer.batch_decode(test_dataloaderx,skip_special_tokens=True)
        pred_decoded = tokenizer.batch_decode(pre,skip_special_tokens=True)
        label_decoded =  tokenizer.batch_decode(test_dataloadery,skip_special_tokens=True)
        
        pred_str = [x  for x in pred_decoded]
        label_str = [[x] for x in label_decoded]
        # pred_list = [x.split()  for x in pred_decoded]
        # label_list = [[x.split()] for x in label_decoded]
        metric_sacrebleu.add_batch(predictions=pred_str, references=label_str)
        # metric_bleu.add_batch(predictions=pred_list, references=label_list)
        if  step==0:
            logging.info(f'x_decoded[:2]:{x_decoded[:2]}')
            logging.info(f'pred_decoded[:2]:{pred_decoded[:2]}')
            logging.info(f'label_decoded[:2]:{label_decoded[:2]}')
            
            
    logging.info('computing score...') 
    sacrebleu_score = metric_sacrebleu.compute()
    # bleu_score = metric_bleu.compute()
    logging.info('%s sacreBLEU : %f',model.name,sacrebleu_score['score'])#TODO:bleu may be wrong cuz max length
    # logging.info('%s BLEU : %f',model.name,bleu_score['bleu'])
    logging.info('%s test loss : %f',model.name,acc/(counter))
    wandb.log({'sacreBLEU'+model.name: sacrebleu_score['score']})
    wandb.log({'test_loss'+model.name: acc/counter})
    # del test_dataloaderx,acc,counter,test_dataloaderx_attn,sacrebleu_score,bleu_score,test_dataloadery,test_dataloadery_attn,ls,pre,x_decoded,pred_decoded,label_decoded,pred_str,label_str,pred_list,label_list
    # gc.collect()
    # torch.cuda.empty_cache()
    model.train()

        

In [12]:
def my_train(epoch, _dataloader, validdataloader, w_model, v_model, architect, A, w_optimizer, v_optimizer, lr_w, lr_v, tot_iter):

    objs_w = AvgrageMeter()
    objs_v_syn = AvgrageMeter()
    objs_v_train = AvgrageMeter()
    objs_v_star_val = AvgrageMeter()
    objs_v_val = AvgrageMeter()
    v_trainloss_acc = 0
    w_trainloss_acc = 0
    # now  train_x is [num of batch, datasize], so its seperate batch for the code below
    wsize = args.train_w_num_points
    synsize = args.train_v_synthetic_num_points
    vsize = args.train_v_num_points
    vtrainsize = vsize+synsize
    vtrainsize_total = train_v_num_points_len+train_v_synthetic_num_points_len
    Asize = args.train_A_num_points
    loader_len = len(_dataloader)
    split_size = [wsize, synsize, vsize, Asize]
    bs = args.batch_size

    logging.info(f"split size:{split_size}")
    for step, batch in enumerate(_dataloader):
        tot_iter[0] += bs
        
        w_model.eval()#!train
        v_model.eval()

        # logging.info(f"GPU mem :{getGPUMem(device)}%")
        train_x = Variable(batch[0], requires_grad=False).to(
            device, non_blocking=False)
        train_x_attn = Variable(batch[1], requires_grad=False).to(
            device, non_blocking=False)
        train_y = Variable(batch[2], requires_grad=False).to(
            device, non_blocking=False)
        train_y_attn = Variable(batch[3], requires_grad=False).to(
            device, non_blocking=False)
        (input_w, input_syn, input_v, input_A_v) = torch.split(train_x, split_size)
        (input_w_attn, input_syn_attn, input_v_attn,
         input_A_v_attn) = torch.split(train_x_attn, split_size)
        (output_w, _, output_v, output_A_v) = torch.split(train_y, split_size)
        (output_w_attn, _, output_v_attn, output_A_v_attn) = torch.split(
            train_y_attn, split_size)
        attn_idx = attn_idx_list[wsize*step:(wsize*step+wsize)]
        # with torch.no_grad():
        #     print('out_old_loss',my_loss2(input_A_v, input_A_v_attn,  output_A_v, output_A_v_attn,v_model))

        # if (args.train_A == 1):
        #     epsilon_w = args.unrolled_w_lr
        #     epsilon_v  = args.unrolled_v_lr
        #     v_star_val_loss = architect.step(input_w,  output_w, input_w_attn, output_w_attn, w_optimizer,
        #                                      input_v, input_v_attn, output_v, output_v_attn, input_syn, input_syn_attn,
        #                                      input_A_v, input_A_v_attn, output_A_v, output_A_v_attn, v_optimizer,
        #                                      attn_idx, lr_w, lr_v)
        #     objs_v_star_val.update(v_star_val_loss, Asize)

        w_model.train()
        w_optimizer.zero_grad()
   
        loss_w = CTG_loss(input_w, input_w_attn, output_w,
                          output_w_attn, attn_idx, A, w_model)
        w_trainloss_acc += loss_w.item()
        loss_w.backward()
        objs_w.update(loss_w.item(), wsize)
        w_optimizer.step()
        # assert False


        # v_optimizer.zero_grad()
        # loss_aug = calc_loss_aug(input_syn, input_syn_attn, w_model, v_model)
        # loss = my_loss2(input_v, input_v_attn, output_v,
        #                 output_v_attn, v_model)
        # v_loss = (args.traindata_loss_ratio*loss +
        #           loss_aug*args.syndata_loss_ratio)
        # v_trainloss_acc += v_loss.item()
        # v_loss.backward()
        # objs_v_syn.update(loss_aug.item(), synsize)
        # objs_v_train.update(loss.item(), vsize)
        # v_optimizer.step()

        # with torch.no_grad():
        #     valloss = my_loss2(input_A_v, input_A_v_attn,  output_A_v, output_A_v_attn,v_model)
        #     objs_v_val.update(valloss.item(), Asize)

        progress = 100*(step)/(loader_len-1)
        if(tot_iter[0] % args.test_num == 0 and tot_iter[0] != 0):
            my_test(_dataloader, model_w, epoch)
            my_test(_dataloader, model_v, epoch)
            logging.info(str(("Attention Weights A : ", A.ReLU(A.alpha))))
            torch.save(model_v,'./model/'+'model_w.pt')#+now+
            torch.save(model_v,'./model/'+'model_v.pt')

        if(tot_iter[0] % args.rep_num == 0 and tot_iter[0] != 0):
            logging.info(f"{progress:5.3}%:\t  W_train_loss:{objs_w.avg:^.7f}\tV_train_syn_loss:{objs_v_syn.avg:^.7f}\tV_train_loss:{objs_v_train.avg:^.7f}\t  V_star_val_loss:{objs_v_star_val.avg:^.7f}\t  V_val_loss:{objs_v_val.avg:^.7f}")
            wandb.log({'W_train_loss': objs_w.avg})
            wandb.log({'V_train_syn_loss': objs_v_syn.avg})
            wandb.log({'V_train_loss': objs_v_train.avg})
            wandb.log({'V_star_val_loss': objs_v_star_val.avg})
            wandb.log({'V_val_loss': objs_v_val.avg})
            objs_v_syn.reset()
            objs_v_train.reset()
            objs_w.reset()
            objs_v_star_val.reset()
            objs_v_val.reset()

    return w_trainloss_acc, v_trainloss_acc


In [13]:
# if(args.valid_begin==1):
#     my_test(valid_dataloader,model_w,-1) #before train
#     my_test(valid_dataloader,model_v,-1)  

tot_iter = [0]
for epoch in range(args.epochs):
    lr_w = scheduler_w.get_lr()[0]
    lr_v = scheduler_v.get_lr()[0]

    logging.info(f"\n\n  ----------------epoch:{epoch},\t\tlr_w:{lr_w},\t\tlr_v:{lr_v},\t\tlr_A:{args.A_lr}----------------")

    w_train_loss,v_train_loss =  my_train(epoch, train_dataloader, valid_dataloader, model_w, model_v,  architect, A, w_optimizer, v_optimizer, lr_w,lr_v,tot_iter)
    
    scheduler_w.step()
    scheduler_v.step()


    logging.info(f"w_train_loss:{w_train_loss},v_train_loss:{v_train_loss}")
    # wandb.log({'w_train_loss': w_train_loss, 'v_train_loss':v_train_loss})



torch.save(model_v,'./model/'+now+'model_w.pt')
torch.save(model_v,'./model/'+now+'model_v.pt')



06/12 11:45:34 PM |	  

  ----------------epoch:0,		lr_w:0.001,		lr_v:0.0,		lr_A:0----------------
06/12 11:45:34 PM |	  split size:[16, 0, 0, 0]
06/12 11:45:40 PM |	   18.2%:	  W_train_loss:5.3267419	V_train_syn_loss:0.0000000	V_train_loss:0.0000000	  V_star_val_loss:0.0000000	  V_val_loss:0.0000000
06/12 11:45:42 PM |	   45.5%:	  W_train_loss:5.0823763	V_train_syn_loss:0.0000000	V_train_loss:0.0000000	  V_star_val_loss:0.0000000	  V_val_loss:0.0000000
06/12 11:45:43 PM |	   72.7%:	  W_train_loss:4.3920604	V_train_syn_loss:0.0000000	V_train_loss:0.0000000	  V_star_val_loss:0.0000000	  V_val_loss:0.0000000
06/12 11:45:49 PM |	  x_decoded[:2]:['translate English to German: Resumption of the session', 'translate English to German: I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.']
06/12 11:45:49 PM |	  pred_decoded[:2]:['Ich habe d

In [ ]:
torch.load('logits.pt')

tensor([[[ 1.6343e-03,  1.6985e-04,  4.6182e-05,  ..., -4.5622e-05,
           4.9963e-05, -7.9150e-05],
         [ 8.2149e-05, -4.7620e-05,  8.2381e-05,  ...,  8.6653e-05,
           1.1960e-04,  3.4265e-06],
         [ 5.5713e-06,  9.0648e-05, -3.2042e-05,  ..., -4.1399e-05,
          -5.0304e-05, -9.8498e-05],
         ...,
         [ 1.6356e-03,  1.6731e-04,  4.5340e-05,  ..., -5.2783e-05,
           4.2435e-05, -8.5164e-05],
         [ 1.6356e-03,  1.6731e-04,  4.5343e-05,  ..., -5.2783e-05,
           4.2434e-05, -8.5163e-05],
         [ 1.6356e-03,  1.6731e-04,  4.5342e-05,  ..., -5.2784e-05,
           4.2434e-05, -8.5163e-05]],

        [[ 1.6345e-03,  1.6776e-04,  4.2936e-05,  ..., -4.4132e-05,
           5.1346e-05, -7.9345e-05],
         [-7.6601e-06, -3.0543e-05, -5.0514e-05,  ...,  1.1752e-05,
          -1.2760e-04,  1.2106e-05],
         [ 2.6243e-05, -3.8717e-05,  8.9031e-05,  ...,  4.1839e-05,
          -1.4667e-05,  3.6620e-05],
         ...,
         [ 1.6364e-03,  1

In [ ]:
m1 = torch.load('main_v.pt')
m1.eval()
m2 = torch.load('unrolled_v.pt')
m2.eval()
''
for k1, k2 in zip(m1.state_dict(), m2.state_dict()):
    v1= m1.state_dict()[k1]
    v2= m2.state_dict()[k2]
    print(k1,k2)
    # print(v1,v2)
    print((abs(v1.data-v2.data)).sum())

FileNotFoundError: [Errno 2] No such file or directory: 'main_v.pt'

In [ ]:
a = torch.ones(1)
a.add(1)


tensor([2.])

In [ ]:
a

tensor([1.])